<a href="https://colab.research.google.com/github/kashmanovav-lab/ML_labs/blob/main/Lab_3_3375_Kashmanova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Для 3 лабораторной работы я выбрала целевой признак — столбец class (1, 2, 3), соответствующий трём сортам пшеницы (он уже имелся).

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, roc_auc_score
)
from sklearn.preprocessing import label_binarize
from itertools import cycle

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt"
columns = [
    'area',
    'perimeter',
    'compactness',
    'length_of_kernel',
    'width_of_kernel',
    'asymmetry_coefficient',
    'length_of_groove',
    'class'
]
df = pd.read_csv(url, sep=r'\s+', names=columns, header=None)

print("Первые 5 строк:")
display(df.head())

print("\nРаспределение классов:")
print(df['class'].value_counts().sort_index())

Первые 5 строк:


,area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_of_groove,class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1



Распределение классов:
class
1    70
2    70
3    70
Name: count, dtype: int64


Импортировали библиотеки, считали данные, вывели первые 5 строк и выяснили, что целевой класс сбалансирован.

In [40]:
X = df.drop(columns=['class'])
y = df['class'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Выполнили подготовку данных: разделение и стандартизацию.

In [41]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

tree = DecisionTreeClassifier(max_depth=5, random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

Обучили модели (kNN и дерево решений).

In [42]:
def evaluate_model(y_true, y_pred, y_proba, model_name, n_classes=3):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    y_true_bin = label_binarize(y_true, classes=np.arange(1, n_classes + 1))
    roc_auc = roc_auc_score(y_true_bin, y_proba, average='macro')

    print(f"\n=== {model_name} ===")
    print(f"Accuracy:   {acc:.4f}")
    print(f"Precision:  {prec:.4f}")
    print(f"Recall:     {rec:.4f}")
    print(f"F1-score:   {f1:.4f}")
    print(f"ROC-AUC:    {roc_auc:.4f}")

y_pred_knn = knn.predict(X_test_scaled)
y_proba_knn = knn.predict_proba(X_test_scaled)

y_pred_tree = tree.predict(X_test)
y_proba_tree = tree.predict_proba(X_test)

evaluate_model(y_test, y_pred_knn, y_proba_knn, "kNN", n_classes=3)
evaluate_model(y_test, y_pred_tree, y_proba_tree, "Decision Tree", n_classes=3)


=== kNN ===
Accuracy:   0.8730
Precision:  0.8721
Recall:     0.8730
F1-score:   0.8713
ROC-AUC:    0.9813

=== Decision Tree ===
Accuracy:   0.9365
Precision:  0.9376
Recall:     0.9365
F1-score:   0.9356
ROC-AUC:    0.9471


Выполнили оценку качества с помощью метрик (Accuracy, Precision, Recall, F1 (частный случай F-measure) и ROC).

In [43]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

knn_cv = KNeighborsClassifier(n_neighbors=5)
knn_scores = cross_val_score(knn_cv, StandardScaler().fit_transform(X), y, cv=cv, scoring='accuracy')

tree_cv = DecisionTreeClassifier(max_depth=5, random_state=42)
tree_scores = cross_val_score(tree_cv, X, y, cv=cv, scoring='accuracy')

print(f"\nКросс-валидация:")
print(f"kNN:            {knn_scores.mean():.4f} ± {knn_scores.std():.4f}")
print(f"Decision Tree:  {tree_scores.mean():.4f} ± {tree_scores.std():.4f}")


Кросс-валидация:
kNN:            0.9238 ± 0.0551
Decision Tree:  0.9095 ± 0.0508


Выполнили кросс-валидацию для объективного сравнения.

**Выводы:**

Обе модели показывают высокую точность. Но при этом kNN немного точнее дерева решений на этом датасете, потому что данные компактны.